In [1]:
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from keras import applications
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

#define dataset path, batch size, and image size
datasetpath = 'dataset'
batch_size = 32
img_size = 128

#initialize VGG16 model for feature extraction
model = applications.VGG16(include_top=False, weights='imagenet')

#ImageDataGenerator instance for train_generator
vgg_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

#create a train_generator using data from the directory of our training set
train_generator = vgg_data_gen.flow_from_directory(
        datasetpath + '/' + 'training',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical', 
        shuffle=False)  

#extract features from the training set
cnn_features = model.predict_generator(train_generator, len(train_generator))

#save the extracted features to a file
filename = 'cnn_features_training'
with open(filename, 'wb') as f:
          np.save(f, cnn_features)

#apply the same thing with validation set
val_generator = vgg_data_gen.flow_from_directory(
        datasetpath + '/' + "validation",
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
cnn_features = model.predict_generator(val_generator, len(val_generator))
filename = 'cnn_features_validation'
with open(filename, 'wb') as f:
          np.save(f, cnn_features)


Using TensorFlow backend.


Found 2197 images belonging to 4 classes.
Found 612 images belonging to 4 classes.


In [2]:
#draw accuracy/loss graph of a given model's history
def draw_graph(History, arg): #arg = acc|loss
    plt.tight_layout()
    plt.plot(History.history[arg])
    val_arg = 'val_' + arg
    plt.plot(History.history[val_arg])
    plt.title('Model '+ arg)
    plt.ylabel(arg)
    plt.xlabel('Epochs')
    plt.legend(['train', 'validation'])
    plt.show()

In [3]:
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical

datagen_top = ImageDataGenerator(rescale=1. / 255)

#get number of training samples and number of classes
nb_train_samples = len(train_generator.filenames)
num_classes = len(train_generator.class_indices)

#save class indices to a file
np.save('class_indices.npy', train_generator.class_indices)

#load extracted features of training set (like X_train)
with open('cnn_features_training', 'rb') as f:
    train_data = np.load(f)

#get the labels of training set (like Y_train)
train_labels = train_generator.classes
#convert class vector to matrix (for categorical_crossentropy)
train_labels = to_categorical(train_labels, num_classes=num_classes)

#get the number of samples of validation set
nb_validation_samples = len(val_generator.filenames)

#load extracted features of the validation set
with open('cnn_features_validation', 'rb') as f:
    validation_data = np.load(f)

#prepare validation labels
validation_labels = val_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)


In [4]:
#create model with only 1 hidden-layer
def create_model(nb_neurons,drop_out = 0.5):
    #initialize a model
    model = Sequential()
    
    #add a Flatten layer and a hidden-layer
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(nb_neurons, activation='relu'))
    model.add(Dropout(drop_out))
    
    #add output layer
    model.add(Dense(num_classes, activation='softmax'))
    
    #specify loss function, optimizer, and metrics of the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
#initialize array of results
Result = []

#apply grid-search-like loops to try multiple combinations of hyperparameters
for drop_out in [0,0.25,0.5]:
    for nb_neurons in [64,128,256]:
        #create model
        model = create_model(nb_neurons, drop_out)
        #apply early_stopping to prevent overfitting
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        #train the model
        history = model.fit(train_data, train_labels,
                            epochs=150,
                            batch_size=batch_size,
                            validation_data=(validation_data, validation_labels),
                            callbacks=[early_stopping]
                            )
        #evaluate model on the validation set
        result = model.evaluate(validation_data, validation_labels, batch_size=batch_size, verbose=1)
        #save the result in the array
        Result.append(result)

Train on 2197 samples, validate on 612 samples
Epoch 1/150
2197/2197 [==============================] - 2s 727us/step - loss: 3.8990 - acc: 0.6909 - val_loss: 2.1532 - val_acc: 0.8186
Epoch 2/150
2197/2197 [==============================] - 1s 451us/step - loss: 1.5521 - acc: 0.8635 - val_loss: 1.6955 - val_acc: 0.8399
Epoch 3/150
2197/2197 [==============================] - 1s 485us/step - loss: 1.1222 - acc: 0.8944 - val_loss: 1.5481 - val_acc: 0.8137
Epoch 4/150
2197/2197 [==============================] - 1s 477us/step - loss: 0.6405 - acc: 0.9235 - val_loss: 0.9032 - val_acc: 0.8268
Epoch 5/150
2197/2197 [==============================] - 1s 504us/step - loss: 0.3249 - acc: 0.9449 - val_loss: 1.0968 - val_acc: 0.8317
Epoch 6/150
2197/2197 [==============================] - 1s 493us/step - loss: 0.1768 - acc: 0.9645 - val_loss: 0.9665 - val_acc: 0.8513
Epoch 7/150
2197/2197 [==============================] - 1s 476us/step - loss: 0.0957 - acc: 0.9841 - val_loss: 0.9898 - val_acc: 0

Epoch 13/150
2197/2197 [==============================] - 2s 872us/step - loss: 0.9074 - acc: 0.9386 - val_loss: 2.0629 - val_acc: 0.8595
Epoch 14/150
2197/2197 [==============================] - 2s 882us/step - loss: 0.8239 - acc: 0.9413 - val_loss: 2.2654 - val_acc: 0.8464
Epoch 15/150
2197/2197 [==============================] - 2s 840us/step - loss: 0.8213 - acc: 0.9390 - val_loss: 2.0134 - val_acc: 0.8611
Epoch 16/150
2197/2197 [==============================] - 2s 848us/step - loss: 0.8109 - acc: 0.9417 - val_loss: 2.3084 - val_acc: 0.8448
Epoch 17/150
2197/2197 [==============================] - 2s 833us/step - loss: 0.8201 - acc: 0.9445 - val_loss: 1.8451 - val_acc: 0.8693
Epoch 18/150
2197/2197 [==============================] - 2s 827us/step - loss: 0.7847 - acc: 0.9431 - val_loss: 1.9330 - val_acc: 0.8676
Epoch 19/150
2197/2197 [==============================] - 2s 821us/step - loss: 0.6877 - acc: 0.9499 - val_loss: 2.0350 - val_acc: 0.8595
Epoch 20/150
2197/2197 [==========

Epoch 27/150
2197/2197 [==============================] - 1s 573us/step - loss: 0.2873 - acc: 0.8985 - val_loss: 0.7665 - val_acc: 0.8578
Epoch 28/150
2197/2197 [==============================] - 1s 552us/step - loss: 0.3229 - acc: 0.8967 - val_loss: 0.8007 - val_acc: 0.8611
Epoch 29/150
612/612 [==============================] - 0s 120us/step
Train on 2197 samples, validate on 612 samples
Epoch 1/150
2197/2197 [==============================] - 2s 1ms/step - loss: 5.5428 - acc: 0.6136 - val_loss: 2.8614 - val_acc: 0.7892
Epoch 2/150
2197/2197 [==============================] - 2s 810us/step - loss: 3.4063 - acc: 0.7569 - val_loss: 2.1394 - val_acc: 0.8301
Epoch 3/150
2197/2197 [==============================] - 2s 837us/step - loss: 2.8225 - acc: 0.7970 - val_loss: 2.5081 - val_acc: 0.8203
Epoch 4/150
2197/2197 [==============================] - 2s 817us/step - loss: 2.6017 - acc: 0.8111 - val_loss: 1.8565 - val_acc: 0.8595
Epoch 5/150
2197/2197 [==============================] - 2s 8

In [6]:
#create model with 2 hidden-layers
def create_model2(nb_neurons,drop_out = 0.5):
    #initialize a model
    model = Sequential()
    #add a Flatten layer and 2 hidden-layers
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(nb_neurons, activation='relu'))
    model.add(Dropout(drop_out))
    model.add(Dense(nb_neurons//2, activation='relu'))
    model.add(Dropout(drop_out))
    #add output layer
    model.add(Dense(num_classes, activation='softmax'))
    
    #specify loss function, optimizer, and metrics of the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
#apply grid-search-like loops to try multiple combinations of hyperparameters
for drop_out in [0,0.25,0.5]:
    for nb_neurons in [128,256,512]:
        #create model
        model = create_model2(nb_neurons, drop_out)
        #apply early_stopping to prevent overfitting
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        #train the model
        history = model.fit(train_data, train_labels,
                            epochs=150,
                            batch_size=batch_size,
                            validation_data=(validation_data, validation_labels),
                            callbacks=[early_stopping]
                            )
        #evaluate model on the validation set
        result = model.evaluate(validation_data, validation_labels, batch_size=batch_size, verbose=1)
        Result.append(result)

Train on 2197 samples, validate on 612 samples
Epoch 1/150
2197/2197 [==============================] - 2s 1ms/step - loss: 3.4407 - acc: 0.6946 - val_loss: 1.9917 - val_acc: 0.7941
Epoch 2/150
2197/2197 [==============================] - 2s 838us/step - loss: 1.3228 - acc: 0.8598 - val_loss: 1.8225 - val_acc: 0.7974
Epoch 3/150
2197/2197 [==============================] - 2s 845us/step - loss: 0.8221 - acc: 0.9058 - val_loss: 1.4569 - val_acc: 0.8366
Epoch 4/150
2197/2197 [==============================] - 2s 940us/step - loss: 0.4712 - acc: 0.9508 - val_loss: 1.8011 - val_acc: 0.7908
Epoch 5/150
2197/2197 [==============================] - 2s 862us/step - loss: 0.3529 - acc: 0.9604 - val_loss: 1.3973 - val_acc: 0.8529
Epoch 6/150
2197/2197 [==============================] - 2s 842us/step - loss: 0.3517 - acc: 0.9663 - val_loss: 1.6031 - val_acc: 0.8464
Epoch 7/150
2197/2197 [==============================] - 2s 831us/step - loss: 0.2448 - acc: 0.9727 - val_loss: 1.4593 - val_acc: 0.8

2197/2197 [==============================] - 2s 855us/step - loss: 6.3457 - acc: 0.5644 - val_loss: 2.7681 - val_acc: 0.7794
Epoch 3/150
2197/2197 [==============================] - 2s 873us/step - loss: 5.0902 - acc: 0.6431 - val_loss: 2.6652 - val_acc: 0.8072
Epoch 4/150
2197/2197 [==============================] - 2s 847us/step - loss: 4.1100 - acc: 0.7037 - val_loss: 2.7781 - val_acc: 0.7908
Epoch 5/150
2197/2197 [==============================] - 2s 849us/step - loss: 3.3621 - acc: 0.7483 - val_loss: 2.6436 - val_acc: 0.8137
Epoch 6/150
2197/2197 [==============================] - 2s 924us/step - loss: 3.1340 - acc: 0.7711 - val_loss: 2.3546 - val_acc: 0.8252
Epoch 7/150
2197/2197 [==============================] - 2s 850us/step - loss: 2.9991 - acc: 0.7833 - val_loss: 2.1734 - val_acc: 0.8333
Epoch 8/150
2197/2197 [==============================] - 2s 882us/step - loss: 2.5683 - acc: 0.8107 - val_loss: 1.8052 - val_acc: 0.8611
Epoch 9/150
2197/2197 [==============================

In [18]:
#show the results in table-like format
i = 0
print('Layers ' +'Drop Out ' + ' Neurons ' + 'Accuracy')
for nb_layers in [1,2]:
    for drop_out in [0,0.25,0.5]:
        for nb_neurons in [128,256,512]:
            print( '%1d & %5.2f & %3d & %5.3f \\\\' % (nb_layers ,drop_out,nb_neurons,Result[i][1]*100))
            print('\hline')
            i = i + 1


Layers Drop Out  Neurons Accuracy
1 &  0.00 & 128 & 84.477 \\
\hline
1 &  0.00 & 256 & 67.647 \\
\hline
1 &  0.00 & 512 & 49.673 \\
\hline
1 &  0.25 & 128 & 84.150 \\
\hline
1 &  0.25 & 256 & 86.438 \\
\hline
1 &  0.25 & 512 & 67.157 \\
\hline
1 &  0.50 & 128 & 87.418 \\
\hline
1 &  0.50 & 256 & 87.255 \\
\hline
1 &  0.50 & 512 & 83.333 \\
\hline
2 &  0.00 & 128 & 86.275 \\
\hline
2 &  0.00 & 256 & 82.353 \\
\hline
2 &  0.00 & 512 & 81.373 \\
\hline
2 &  0.25 & 128 & 85.784 \\
\hline
2 &  0.25 & 256 & 84.150 \\
\hline
2 &  0.25 & 512 & 61.601 \\
\hline
2 &  0.50 & 128 & 84.150 \\
\hline
2 &  0.50 & 256 & 82.680 \\
\hline
2 &  0.50 & 512 & 68.954 \\
\hline
